<a href="https://colab.research.google.com/github/swicklund/hackathon/blob/main/MLModel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import os

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import fastbook as fastai
import pandas as pd
from pandas.io.json import json_normalize
import seaborn as sns
import tensorflow as tf
import json
import zipfile
from io import BytesIO
import torch
import math
from google.colab import files
from google.colab import auth
from datetime import datetime, timedelta

fastai.setup_book()
from fastai import tabular

In [2]:
auth.authenticate_user()

In [3]:
!gcloud config set project {'hackathon-352519'}

Updated property [core/project].


In [4]:
!gsutil cp gs://trains_data/* /tmp/.

Copying gs://trains_data/train-data_all-train-events.zip...
\ [1 files][ 57.1 MiB/ 57.1 MiB]                                                
Operation completed over 1 objects/57.1 MiB.                                     


In [5]:
trains = {}
i = 1
prevLoco = ''
with zipfile.ZipFile("/tmp/train-data_all-train-events.zip", "r") as f:
    for name in f.namelist():
        if name.endswith('.json'):
            with f.open(name) as zd:
                data = json.load(zd)
                lastTrainReporting = data['lastTrainReporting']
                currentLoco = lastTrainReporting['ptcLeadLocomotiveId']
                # prevLoco != currentLoco and 
                if ("trainActualPositionList" in lastTrainReporting and len(lastTrainReporting['trainActualPositionList']) > 0): 
                    trainActualPositionList = lastTrainReporting['trainActualPositionList']
                    trains[i] = pd.json_normalize(lastTrainReporting)
                    i = i + 1
                    if (i % 1000 == 0):
                      current_time = datetime.now().strftime("%H:%M:%S")
                      print(i, " ", current_time)
                    prevLoco = currentLoco
                            

1000   16:32:55
2000   16:33:07
3000   16:33:12
4000   16:33:18
5000   16:33:23
6000   16:33:28
7000   16:33:34
8000   16:33:40
9000   16:33:45
10000   16:33:51


In [6]:
dfTrainsAll = pd.concat(trains.values())
dfTrainsAll["milepost"] = dfTrainsAll.apply(lambda row: math.trunc(row['lastReportedPosition.milepostLocation.milepost.milepostNumber']), axis=1)

In [7]:
del trains

In [8]:
print(dfTrainsAll.info())
print(dfTrainsAll.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10054 entries, 0 to 0
Data columns (total 45 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   ptcLeadLocomotiveId                                                        10054 non-null  object 
 1   maximumTrainSpeed                                                          10053 non-null  float64
 2   loadedCarCount                                                             10053 non-null  float64
 3   emptyCarCount                                                              10053 non-null  float64
 4   trainLength                                                                10053 non-null  float64
 5   trailingTonnage                                                            10053 non-null  float64
 6   positionHistoryList                                       

In [9]:
dfTrainsLearn = dfTrainsAll
dfTrainsLearn['lastReportedPosition.milepostLocation.trackName'] = dfTrainsLearn['lastReportedPosition.milepostLocation.trackName'].astype('category')
dfTrainsLearn['lastReportedPosition.milepostLocation.trackName'] = dfTrainsLearn['lastReportedPosition.milepostLocation.trackName'].cat.set_categories(['WEST MAIN', 'MAIN', 'MT 2', 'MT 1', 'WESTXO', 'XO', 'EASTXO','YD LD', 'EAST MAIN', 'TO_N_EAST MAIN', 'TO_S_UP_MAINLINE', 'TO_S_CS JUNCTION', 'S_STARKS', 'TO_S_KCS UP PRT CONN', 'TO_N_LUCAS', 'S_LUCAS'])
dfTrainsLearn['lastReportedSpeedMPH'] = dfTrainsLearn.apply(lambda row: row['positionHistoryList'][0]['speedMPH'], axis=1)
dfTrainsLearn['symbolType'] = dfTrainsLearn.apply(lambda row: row['trainId.trainSymbol'][:1], axis=1)
dfTrainsLearn['symbolType'] = dfTrainsLearn['symbolType'].astype('category')
dfTrainsLearn['symbolType'] = dfTrainsLearn['symbolType'].cat.set_categories(['M', 'I', 'Z', 'C', 'W', 'E', 'U'])


In [28]:
def getWorkLocations(dfTrainsLocation):
  if len(dfTrainsLocation['aotuTrainData.routeLocationList']) == 0:
    return None
  aotuRoute=pd.json_normalize(dfTrainsLocation['aotuTrainData.routeLocationList']) 
  eventLocations = aotuRoute[aotuRoute['activityList'].map(len) > 1]
  temp = eventLocations['milepost.milepostNumber'].unique()
  if len(temp) == 0:
    return None
  return ','.join(['%.2f' % num for num in temp])

In [25]:
#getWorkLocations(dfTrainsLearn.iloc[301])

'202.20'

In [13]:
#dfTrainsLearn.loc[dfTrainsLearn['aotuTrainData.routeLocationList'].map(len)==0]

In [29]:
#workLocations = getWorkLocations(dfTrainsLearn.iloc[101])
dfTrainsLearn['workLocations'] = dfTrainsLearn.apply(lambda row: getWorkLocations(row), axis=1)

In [30]:
dfTrainsLearn['workLocations'].unique()

array([None, '202.20', '258.40,202.20', '202.20,258.40', '202.20,258.40,321.50', '202.92,202.20', '225.10,202.20', '240.90,202.20'], dtype=object)

We can make this more clever later, but for now this seems the most sensible shortcut

In [31]:
dfTrainsLearn['locomotiveCount'] = dfTrainsLearn.apply(lambda row: len(row['locomotiveList']),axis = 1)

In [32]:
dfTrainsLearnActuals = dfTrainsLearn.apply(lambda row: pd.json_normalize(row['trainActualPositionList']), axis=1)

In [33]:
dfTrainsLearnActuals.iloc[1]

,positionTime,speedMPH,milepostLocation.subdivisionId,milepostLocation.trackName,milepostLocation.milepost.milepostNumber
0,2022-04-21T23:34:11Z,-10,106,WEST MAIN,766.36030
1,2022-04-21T23:36:11Z,-10,106,MAIN,766.01776
2,2022-04-21T23:38:11Z,-9,106,MAIN,765.68665
3,2022-04-21T23:40:11Z,-8,106,MAIN,765.38310
4,2022-04-21T23:42:11Z,-9,106,MAIN,765.07520


In [34]:
def findNextSpeed(trainsEntry, actualsEntry):
  milepost = trainsEntry['lastReportedPosition.milepostLocation.milepost.milepostNumber']
  #print(milepost)
  value  = actualsEntry.loc[actualsEntry['milepostLocation.milepost.milepostNumber'] > milepost+1]
  if (len(value) > 0):
    return value.min()['speedMPH']
  value  = actualsEntry.loc[actualsEntry['milepostLocation.milepost.milepostNumber'] < milepost-1]
  if (len(value) > 0):
    return value.max()['speedMPH']
  return None

In [35]:
dfTrainsLearn['nextSpeed'] = dfTrainsLearn['lastReportedPosition.speedMPH']
i = 0;
for index, trainsEntry in dfTrainsLearn.iterrows():
  actualsEntry = dfTrainsLearnActuals.iloc[i]
  nextSpeed = findNextSpeed(trainsEntry, actualsEntry)
  if (nextSpeed != None):
    trainsEntry['nextSpeed'] = nextSpeed
  if (i % 1000 == 0):
    print(i)
  i = i + 1

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  if __name__ == '__main__':


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [19]:
# dfTrainsLearn = dfTrainsLearn.drop(['hour'], axis=1)

Drop some values we are not using at the moment not needed

In [37]:
dfTrainsLearn = dfTrainsLearn.drop(columns=['aotuTrainData.tenantCarrier',
                                            'lastReportedPosition.precedingSignalState.currentStateTime', 
                                            'lastReportedPosition.nextSignalState.previousSignalState', 
                                            'lastReportedPosition.nextSignalState.currentStateTime', 
                                            'positionHistoryList',
                                            'locomotiveList',
                                            'subdivisionList',
                                            'trainActualPositionList',
                                            'trainEstimatedPositionList',
                                            'aotuTrainData.routeLocationList'])

In [36]:
tabular.core.make_date(dfTrainsLearn, 'trainId.trainDate')
tabular.core.make_date(dfTrainsLearn, 'lastReportedPosition.positionTime')
tabular.core.add_datepart(dfTrainsLearn, 'lastReportedPosition.positionTime')
# tabular.core.add_elapsed_times(dfTrainsLearn, 'lastReportedPosition.positionTime')


,ptcLeadLocomotiveId,maximumTrainSpeed,loadedCarCount,emptyCarCount,trainLength,trailingTonnage,positionHistoryList,locomotiveList,subdivisionList,trainEstimatedPositionList,trainActualPositionList,trainId.trainSCAC,trainId.trainSection,trainId.trainSymbol,trainId.trainDate,lastReportedPosition.speedMPH,lastReportedPosition.milepostLocation.subdivisionId,lastReportedPosition.milepostLocation.trackName,lastReportedPosition.milepostLocation.milepost.milepostNumber,aotuTrainData.priorityCode,aotuTrainData.categoryCode,aotuTrainData.authorityRangeCode,aotuTrainData.ownershipCode,aotuTrainData.tenantCarrier,aotuTrainData.totalCarWeight,aotuTrainData.totalCarLength,aotuTrainData.totalLocomotiveWeight,aotuTrainData.totalLocomotiveLength,aotuTrainData.actualTonsPerAxle,aotuTrainData.heavy143TonCarCount,aotuTrainData.heavy158TonCarCount,aotuTrainData.over158TonCarCount,aotuTrainData.highWideCarCount,aotuTrainData.eventCode,aotuTrainData.routeLocationList,lastReportedPosition.precedingSignalState.milepostLocation.milepostNumber,lastReportedPosition.precedingSignalState.previousSignalState,lastReportedPosition.precedingSignalState.currentStateTime,lastReportedPosition.precedingSignalState.signalState,lastReportedPosition.nextSignalState.milepostLocation.milepostNumber,lastReportedPosition.nextSignalState.signalState,lastReportedPosition.nextSignalState.previousSignalState,lastReportedPosition.nextSignalState.currentStateTime,milepost,lastReportedSpeedMPH,symbolType,workLocations,locomotiveCount,nextSpeed,lastReportedPosition.positionTimeYear,lastReportedPosition.positionTimeMonth,lastReportedPosition.positionTimeWeek,lastReportedPosition.positionTimeDay,lastReportedPosition.positionTimeDayofweek,lastReportedPosition.positionTimeDayofyear,lastReportedPosition.positionTimeIs_month_end,lastReportedPosition.positionTimeIs_month_start,lastReportedPosition.positionTimeIs_quarter_end,lastReportedPosition.positionTimeIs_quarter_start,lastReportedPosition.positionTimeIs_year_end,lastReportedPosition.positionTimeIs_year_start,lastReportedPosition.positionTimeElapsed
0,UP2010,50.0,61.0,92.0,9282.0,10992.0,"[{'positionTime': '2022-04-21T23:31:08Z', 'speedMPH': -12, 'milepostLocation': {'subdivisionId': 106, 'trackName': 'WEST MAIN', 'milepost': {'milepostNumber': 766.9304}}}, {'positionTime': '2022-04-21T23:29:54Z', 'speedMPH': -10, 'milepostLocation': {'subdivisionId': 595, 'trackName': 'MT 1', 'milepost': {'milepostNumber': 279.0858}}}, {'positionTime': '2022-04-21T23:27:47Z', 'speedMPH': -5, 'milepostLocation': {'subdivisionId': 595, 'trackName': 'MT 1', 'milepost': {'milepostNumber': 279.3492}}}, {'positionTime': '2022-04-21T23:27:06Z', 'speedMPH': -5, 'milepostLocation': {'subdivisionId'...","[{'locomotiveId': 'UP2010', 'positionOnTrain': 1, 'locomotiveWeight': 210, 'locomotiveLength': 73, 'locomotiveHorsepower': 4400}, {'locomotiveId': 'UP5597', 'positionOnTrain': 2, 'locomotiveWeight': 207, 'locomotiveLength': 73, 'locomotiveHorsepower': 4400}]","[{'subdivisionId': 595}, {'subdivisionId': 538}, {'subdivisionId': 461}, {'subdivisionId': 605}, {'subdivisionId': 599}]","[{'positionTime': '2022-04-21T23:31:08Z', 'speedMPH': -12, 'milepostLocation': {'subdivisionId': 106, 'trackName': 'WEST MAIN', 'milepost': {'milepostNumber': 766.9304}}}, {'positionTime': '2022-04-21T23:33:08Z', 'speedMPH': -19, 'milepostLocation': {'subdivisionId': 106, 'trackName': 'WEST MAIN', 'milepost': {'milepostNumber': 766.4076}}}, {'positionTime': '2022-04-21T23:35:08Z', 'speedMPH': -25, 'milepostLocation': {'subdivisionId': 106, 'trackName': 'WEST MAIN', 'milepost': {'milepostNumber': 765.6378}}}, {'positionTime': '2022-04-21T23:37:08Z', 'speedMPH': -30, 'milepostLocation': {'su...","[{'positionTime': '2022-04-21T23:33:08Z', 'speedMPH': -10, 'milepostLocation': {'subdivisionId': 106, 'trackName': 'WEST MAIN', 'milepost': {'milepostNumber': 766.5422}}}, {'positionTime': '2022-04-21T23:35:08Z', 'speedMPH': -10, 'milepostLocation': {'subdivisionId': 106, 'trackName': 'WES

In [38]:
print(dfTrainsLearn.shape)
print(dfTrainsLearnActuals.shape)

print(dfTrainsLearn.info())
print(dfTrainsLearn.iloc[100])

(10054, 52)
(10054,)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10054 entries, 0 to 0
Data columns (total 52 columns):
 #   Column                                                                     Non-Null Count  Dtype         
---  ------                                                                     --------------  -----         
 0   ptcLeadLocomotiveId                                                        10054 non-null  object        
 1   maximumTrainSpeed                                                          10053 non-null  float64       
 2   loadedCarCount                                                             10053 non-null  float64       
 3   emptyCarCount                                                              10053 non-null  float64       
 4   trainLength                                                                10053 non-null  float64       
 5   trailingTonnage                                                            10053 non-null  f

# Use Decision Tree to predict speed in 1 mile

In [40]:
procs = [tabular.core.Categorify, tabular.core.FillMissing]

In [41]:
print (dfTrainsLearn.groupby(['lastReportedPosition.positionTimeDayofyear'])['lastReportedPosition.positionTimeDayofyear'].count())


lastReportedPosition.positionTimeDayofyear
90      124
91      271
92       79
93      106
95      109
96      171
97      195
98      256
99      168
100     267
101      30
103     277
104     180
105     433
106     739
107     752
108     607
109     517
110     545
111     600
112    1032
113     735
114     343
115     262
116     198
117      18
118       5
120     142
121      45
122     247
123     214
124     237
125     150
Name: lastReportedPosition.positionTimeDayofyear, dtype: int64


In [42]:
cond = ( dfTrainsLearn['lastReportedPosition.positionTimeDayofyear'] < 121 )
trainIdx = np.where(cond)[0]
validIdx = np.where(~cond)[0]
splits = (list(trainIdx), list(validIdx))

In [43]:
dep_var = 'nextSpeed'

In [44]:
cont,cat = tabular.core.cont_cat_split(dfTrainsLearn, 1, dep_var=dep_var)

In [45]:
to = tabular.core.TabularPandas(dfTrainsLearn, procs, cat, cont, y_names=dep_var, splits=splits)

#OLD CODE

In [ ]:
def formatTrain(dfTrain):
  actuals = pd.json_normalize(dfTrain['trainActualPositionList'])
  cols = list(actuals.columns)
  dfx = (actuals[cols[0:2] + cols[3:5]])
  dfx = dfx.rename(columns={"speedMPH" : "speedMPH", "milepostLocation.trackName":"trackName", "milepostLocation.milepost.milepostNumber": "milepostNumber" })
  est = pd.json_normalize(dfTrain['trainEstimatedPositionList'])  
  beop = est.rename(columns={"speedMPH" : "forecastSpeedMPH", "milepostLocation.trackName":"forecastTrackName", "milepostLocation.milepost.milepostNumber": "forecastMilepostNumber" })
  joined = beop.join(dfx, lsuffix='_beop', rsuffix='_actual')
  dn = joined.dropna()
  dn = dn.loc[(dn['speedMPH'] != 0)]
  delta = abs(dn["speedMPH"] - dn["forecastSpeedMPH"]);
  #dn = dn.apply(lambda x: abs(x) if x.name=='speedMPH' or  x.name=='forecastSpeedMPH' else x)
  dn["delta"] = delta
  try:
    dn['milepost'] = dn.apply(lambda row: math.trunc(row['milepostNumber']), axis=1) 
    dn['millis'] = dn.apply(lambda row: ((datetime.strptime(row['positionTime_actual'], "%Y-%m-%dT%H:%M:%SZ"))  - datetime(1970, 1, 1)) // timedelta(milliseconds=1), axis=1)
  except ValueError:
    print (ValueError)
    print("Oops!  invalid format", dn)
  dn['trainId'] = dfTrain['trainId.trainSymbol']+" "+dfTrain['trainId.trainDate']
  dn = dn.drop(columns=['milepostLocation.subdivisionId'])
  # dn = dn.reset_index()
  return dn

In [ ]:
def convertTrains(trainSet): 
  trainsList = []
  for index, train in trainSet.iterrows():
    # print(train['trainId.trainSymbol'])
    trainsList.append(formatTrain(train))
  trains = pd.concat(trainsList)
  trains = trains.sort_values(['trainId','positionTime_actual'])
  trains = trains.drop_duplicates(subset=['trainId', 'milepost'], keep='last')
  trains = trains.reset_index()
  return trains

In [ ]:
trains = convertTrains(dfTrains)

In [ ]:
def getTrain(trains, symbol):
  return trains.loc[trains['trainId'].str.startswith(symbol)]

In [ ]:
getTrain(trains, 'MALNP')

Compute the Change of Velocity entering each milepost segment

In [ ]:
def computeSpeedChange(trains):
  trains['speedChange'] = trains["speedMPH"]
  prevTrain = ''
  for i in range(0, len(trains)):
    if trains.loc[i, "trainId"] == prevTrain :
      trains.loc[i, "speedChange"] = trains.loc[i, "speedMPH"] - trains.loc[i-1, "speedMPH"]
    else :  
      trains.loc[i, "speedChange"] = 0.0
    prevTrain = trains.loc[i, "trainId"]
  return trains

In [ ]:
trains = computeSpeedChange(trains)
trains

In [ ]:
mps = trains['milepost'].unique()
mps.sort()
mps

In [ ]:
print(trains.iloc[0])

In [ ]:
def printAllReadings(trainSet, trainSymbol):
  with pd.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.width', 1000,
                        'display.precision', 3,
                        ):
    vals = trainSet.loc[trainSet['trainId'] == trainSymbol]
    #vals= vals.sort_values('positionTime_actual')
    print(vals[['index', 'positionTime_actual', 'milepost', 'milepostNumber','forecastSpeedMPH', 'speedMPH', 'speedChange']])

In [ ]:
printAllReadings(trains, 'ITAG3 2022-04-20')

In [ ]:
dfTrains.iloc[3]["trainEstimatedPositionList"]                                                   

In [ ]:
dfTrains.iloc[3]

In [ ]:
def plotTrain(trainData, trainId):
  ax = plt.gca()
  trainDF = trainData.loc[trainData['trainId'] == trainId]
  trainDF = trainDF.reset_index();
  trainsDF = trainDF.sort_values('positionTime_actual')
  trainDF = trainDF.apply(lambda x: abs(x) if x.name=='speedMPH' or  x.name=='forecastSpeedMPH' else x)  
  trainDF.plot(x = 'milepostNumber', y="speedMPH", use_index=True, kind="scatter", ax=ax, figsize=[24,10])
  trainDF.plot(x = 'milepostNumber', y="forecastSpeedMPH", use_index=True, kind="scatter", color="red", ax=ax, figsize=[24,10])
  trainDF.plot(x = 'milepostNumber', y="delta", use_index=True, kind="line", color="green", ax=ax, figsize=[24,10])
  # plt.gcf().autofmt_xdate()
  (t,b)= ax.get_xlim()  ## Added for clarity
  if t < b:
    ax.set_xlim((b,t))
  plt.title(trainId)
  plt.show()

In [ ]:
def printTrains(trains):
  trainIdsList = trains['trainId'].unique()
  print(trainIdsList.size)
  print(trainIdsList)

In [ ]:
printTrains(trains)

In [ ]:
plotTrain(trains, trainIdsList[14])

In [ ]:
trains.head()

In [ ]:
indexedMean = (trains[["index","delta"]]).groupby('index').mean()
indexedMean.plot( y="delta", kind="line", color="green", figsize=[24,10])
plt.title("Mean Error")

In [ ]:
meanSquaredError = math.sqrt((indexedMean['delta']**2).sum())
print("Mean Squared Error: ", meanSquaredError)

In [ ]:
indexedMeanByMilepost = (trains[["milepost","delta"]]).groupby('milepost').mean()
indexedMeanByMilepost = indexedMeanByMilepost.dropna().reset_index()
print(indexedMeanByMilepost)
indexedMeanByMilepost.plot(y="delta", use_index=True, kind="line", color="red", figsize=[24,10])
# labels= indexedMeanByMilepost['milepost'].values
plt.xlabel('milepost')
plt.title("Mean Error by Milepost")

Begin Training New Model<br>
We shall govern *increase in speed* to be a maximimum of the standard deviation for the milepost area. 


In [ ]:
dfTrainIds = dfTrains[['trainId.trainSymbol','trainId.trainDate','lastReportedPosition.positionTime']]
dfTrainIds = dfTrainIds.drop_duplicates(subset=['trainId.trainSymbol','trainId.trainDate'], keep='first')
dfTrainIds = dfTrainIds.sort_values(['lastReportedPosition.positionTime'])
dfTrainIds

In [ ]:
count = dfTrainIds.shape[0]
trainingCount = count // 4 * 3
trainingSet = dfTrains.head(trainingCount)
testingSet = dfTrains.tail(count - trainingCount)

In [ ]:
trainSpeeds = trains.sort_values(['trainId', 'positionTime_actual'])
trainSpeeds = trainSpeeds.drop_duplicates(subset=['trainId'], keep='first')
trainSpeeds = trainSpeeds.sort_values(['positionTime_actual'])
trainingSpeedsCount = trainSpeeds.shape[0] // 4 * 3
trainingSpeedsIdSet = trainSpeeds.head(trainingSpeedsCount)
testingSpeedsIdSet = trainSpeeds.tail(trainSpeeds.shape[0] - trainingSpeedsCount)
trainingSpeeds = pd.merge(trains, trainingSpeedsIdSet["trainId"])
trainingSpeeds

In [ ]:
def minMaxMean(trainsSet): 
  vals = trainsSet
  speedChangeMean = {"speedChangeMean": trainsSet[["speedChange"]].mean()}
  speedChangeMax = {"speedChangeMax" : trainsSet[["speedChange"]].max()}
  speedChangeMin = {"speedChangeMin" : trainsSet[["speedChange"]].min()}
  speedChangeStd = {"speedChangeStd" : trainsSet[["speedChange"]].std()}
  return [speedChangeMean, speedChangeMax, speedChangeMin, speedChangeStd, vals]


In [ ]:
mps = trains['milepost'].unique()
mps.sort()
mps

In [ ]:
mpSpeedsPositive = {}
for mp in mps:
  mpTrains = trainingSpeeds.loc[trainingSpeeds['milepost'].between(mp - 1, mp +1)]
  mpSpeedsPositive[mp] = minMaxMean(mpTrains.loc[mpTrains.speedMPH > 0])
mpSpeedsPositive[203.0]


In [ ]:
mpSpeedsNegative = {}
for mp in mps:
  mpTrains = trainingSpeeds.loc[trainingSpeeds['milepost'].between(mp - 1, mp +1)]
  mpSpeedsNegative[mp] = minMaxMean(mpTrains.loc[mpTrains.speedMPH < 0])
mpSpeedsNegative[203.0]


In [ ]:
testTrains = convertTrains(testingSet)
printTrains(testTrains)

In [ ]:
plotTrain(testTrains, 'MCHCB 2022-04-25')

In [ ]:
computeSpeedChange(testTrains)
testTrains

In [ ]:
printAllReadings(testTrains, "MCHCB 2022-04-25")

I don't actually think this is FIXABLE...